# German Credit Risk 

Notebook using German Credit Risk Dataset from https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data and methods from paper: "Explainable AI for Credit Assessment in Banks" (https://www.mdpi.com/1911-8074/15/12/556). Some data preparation is based on https://github.com/alicenkbaytop/German-Credit-Risk-Classification/blob/master/german-credit-risk-classification.ipynb. 

Note that text for explanation in this notebook is for the most part copied/adapted from the paper above.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

# Dataset


In [2]:
df = pd.read_csv('german_credit_data.csv')

In [3]:
print(df.head())

   Unnamed: 0  Age     Sex  Job Housing Saving accounts Checking account  \
0           0   67    male    2     own             NaN           little   
1           1   22  female    2     own          little         moderate   
2           2   49    male    1     own          little              NaN   
3           3   45    male    2    free          little           little   
4           4   53    male    2    free          little           little   

   Credit amount  Duration              Purpose  Risk  
0           1169         6             radio/TV  good  
1           5951        48             radio/TV   bad  
2           2096        12            education  good  
3           7882        42  furniture/equipment  good  
4           4870        24                  car   bad  


In [4]:
df["Monthly pay"] = (df["Credit amount"] / df["Duration"])
df["Credit amount^2"] = df["Credit amount"]**2